In [ ]:
# abrir musicar e pré-processar melspectogram
import pandas as pd
import librosa as mimosa
import audioread
import os
import matplotlib as plt
import seaborn as sns
import numpy as np

musicGender = os.listdir("musics") # returns list
genderlabel = []
musicData = []

for gender in musicGender:
    genderMusics = os.listdir('./musics/{}'.format(gender))
    for music in genderMusics:
        genderlabel.append(gender)
        
        amplitude, sr = mimosa.load('./musics/{}'.format(gender)+'/{}'.format(music), offset=15, duration=15)

        melSpec = mimosa.feature.melspectrogram(y = amplitude, sr=sr, n_mels=128, fmax=8000)    
        melSpec_to_db = mimosa.power_to_db(melSpec, ref=np.max)
        resized_melSpec = np.resize(melSpec_to_db, (melSpec_to_db.shape[0], 1293))
        
        musicData.append(resized_melSpec)

mfcc_specData=np.stack(musicData)
musicData=np.array(musicData)
genderlabel=np.array(genderlabel)


In [ ]:
# fazer encoding das categorias
import sklearn.preprocessing as preProSkL

labelEncoder = preProSkL.LabelEncoder()
labelEncoded = labelEncoder.fit(genderlabel)
labelEncodedTrans = labelEncoded.transform(genderlabel)
# inverse_transform to revert to normal

In [ ]:
# normalizar dados por media
# não usado, mas pode ser comentado

# from sklearn.preprocessing import StandardScaler

# sc_X=StandardScaler()
# mfcc_specData = sc_X.fit_transform(mfcc_specData)

# mfcc_specData.head()


# mean = np.mean(musicData, axis=-1, keepdims=True)
# std = np.std(musicData, axis=-1, keepdims=True)


# Normalize the array by subtracting the mean and dividing by the standard deviation
# normalized_data = (musicData - mean) / std

# print(len(normalized_data))


In [ ]:
# Normalização Min Max

indexMin=0
indexMax = 100

normalized_data = []
numCategories = int( len(labelEncodedTrans)/ 100)


for i in range(numCategories):

    max = mfcc_specData[indexMin:indexMax].max()
    min = mfcc_specData[indexMin:indexMax].min()

    h = 0
    for music in musicData[indexMin:indexMax]:

        xScaled = (music - min) / (max - min)
        normalized_data.append(xScaled)

    indexMin += 100
    indexMax += 100  

normalized_data=np.array(normalized_data)

print(len(normalized_data))


In [27]:
# modelo usado, atravé da separação dos dados com k-folds

from sklearn.model_selection import KFold
from tensorflow import keras
import matplotlib.pyplot as pyplot

acc_per_fold = []
loss_per_fold = []


predicted_classes = np.empty([0], dtype=int)

input_shape=(normalized_data.shape[1],normalized_data.shape[2],1)
fold_no = 1

kf = KFold(n_splits=3, random_state=42, shuffle=True)
for train, test in kf.split(normalized_data):
    model=keras.Sequential()

    #1st layer
    model.add(keras.layers.Conv2D(6,(3,3),activation="relu",input_shape=input_shape))
    model.add(keras.layers.MaxPool2D((3,3),strides=(2,2),padding="same")) 
    model.add(keras.layers.BatchNormalization())

    # #2nd layer
    # model.add(keras.layers.Conv2D(8,(3,3),activation="relu"))
    # model.add(keras.layers.MaxPool2D((3,3),strides=(2,2),padding="same")) #pooling
    # model.add(keras.layers.BatchNormalization())

    # #3rd layer
    # model.add(keras.layers.Conv2D(8,(1,1),activation="relu"))
    # model.add(keras.layers.MaxPool2D((1,1),strides=(1,1),padding="same")) #pooling
    # model.add(keras.layers.BatchNormalization())


    #flatten the output
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(8,activation="relu"))
    # model.add(keras.layers.Dropout(0.3))

    #output layer
    model.add(keras.layers.Dense(3,activation="softmax"))


    optimizer=keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=['accuracy'])
    
    
    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
        
    # normalized_data[train] = normalized_data[train].reshape((-1, normalized_data[train].shape[1], normalized_data[train].shape[2], 1))
    history = model.fit(normalized_data[train],labelEncodedTrans[train],epochs=25, validation_data=(normalized_data[test], labelEncodedTrans[test]))

    testError, testAccuracy = model.evaluate(normalized_data[test], labelEncodedTrans[test])
    print("Accuracy on test set is:{}".format(testAccuracy))


    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {testError}; {model.metrics_names[1]} of {testAccuracy*100}%')
    acc_per_fold.append(testAccuracy * 100)
    loss_per_fold.append(testError)

    # Increase fold number
    fold_no = fold_no + 1

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
    print('------------------------------------------------------------------------')
    print('Average scores for all folds:')
    print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
    print(f'> Loss: {np.mean(loss_per_fold)}')
    print('------------------------------------------------------------------------')
    

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/25
7/7 [==============================] - 3s 411ms/step - loss: 0.8888 - accuracy: 0.6550 - val_loss: 1.6127 - val_accuracy: 0.3000
Epoch 2/25
7/7 [==============================] - 3s 396ms/step - loss: 0.4816 - accuracy: 0.7650 - val_loss: 1.2733 - val_accuracy: 0.3400
Epoch 3/25
7/7 [==============================] - 3s 392ms/step - loss: 0.3707 - accuracy: 0.8350 - val_loss: 1.4381 - val_accuracy: 0.3300
Epoch 4/25
7/7 [==============================] - 3s 403ms/step - loss: 0.3037 - accuracy: 0.8850 - val_loss: 1.0607 - val_accuracy: 0.3600
Epoch 5/25
7/7 [==============================] - 3s 398ms/step - loss: 0.2409 - accuracy: 0.9100 - val_loss: 1.1571 - val_accuracy: 0.3900
Epoch 6/25
7/7 [==============================] - 3s 395ms/step - loss: 0.2146 - accuracy: 0.8700 - val_loss: 1.1718 - val_accuracy: 0.3900
Epoch 7/25
7/7 [==============================] - 3s 400ms/step

In [ ]:
# separação antiga dos dados de treino e teste

# import sklearn.model_selection as modelSelec

# X_train, X_test, y_train, y_test = modelSelec.train_test_split(normalized_data, labelEncodedTrans, test_size=0.25, random_state=42)

# # X_train, X_test, y_val, y_val = modelSelec.train_test_split(mfcc_specData, labelEncodedTrans, test_size=0.5, random_state=42)
# X_test,X_validation,y_test,y_validation=modelSelec.train_test_split(X_test,y_test,test_size=0.2, random_state=42)


In [ ]:
# plot do mel spectogram

import matplotlib.pyplot as pyplot



fig, ax = pyplot.subplots()
img = mimosa.display.specshow(normalized_data[100], x_axis='time', y_axis='mel', sr=sr, fmax=8000, ax=ax)
fig.colorbar(img, ax=ax, format='%+2.0f dB')

ax.set(title='Mel-frequency spectrogram')

pyplot.savefig('0.png')

  

In [ ]:
# criados dos plots através dos dados normalizados 

# import matplotlib.pyplot as pyplot

# musicCounter = 0

# path = './musicsPlts'
# if not os.path.exists(path):
#     os.mkdir(path)

# for gender in musicGender:
#     genderPath = '{}'.format(path) + '/{}'.format(gender)
#     if not os.path.exists(genderPath):
#         os.mkdir(genderPath)

# for index in range(len(mfcc_specData)):
#     fig, ax = pyplot.subplots()
#     img = mimosa.display.specshow(mfcc_specData[index], x_axis='time', y_axis='mel', sr=sr, fmax=8000, ax=ax)
#     fig.colorbar(img, ax=ax, format='%+2.0f dB')

#     ax.set(title='Mel-frequency spectrogram')

#     if index % 100 == 0:
#         musicCounter = 0
    
#     pyplot.savefig('{}'.format(path) + '/{}'.format(genderlabel[index]) + '/{}'.format(musicCounter) + '.png')

#     musicCounter +=1




In [ ]:
# formato dos dados para modelo antigo

print(X_train.shape)
input_shape=(X_train.shape[1],X_train.shape[2],1)
print(X_train.shape)
print(input_shape)


In [ ]:
# modelo antigo testado com variantes

from tensorflow import keras

model=keras.Sequential()
#1st layer
model.add(keras.layers.Conv2D(24,(3,3),activation="relu",input_shape=input_shape))
model.add(keras.layers.MaxPool2D((3,3),strides=(2,2),padding="same")) 
model.add(keras.layers.BatchNormalization())

#2nd layer
model.add(keras.layers.Conv2D(24,(3,3),activation="relu"))
model.add(keras.layers.MaxPool2D((3,3),strides=(2,2),padding="same")) #pooling
model.add(keras.layers.BatchNormalization())

#3rd layer
model.add(keras.layers.Conv2D(24,(2,2),activation="relu"))
model.add(keras.layers.MaxPool2D((2,2),strides=(2,2),padding="same")) #pooling
model.add(keras.layers.BatchNormalization())


#flatten the output
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(20,activation="relu"))
model.add(keras.layers.Dropout(0.2))

#output layer
model.add(keras.layers.Dense(10,activation="softmax"))

optimizer=keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer,
    loss="sparse_categorical_crossentropy",
    metrics=['accuracy'])


In [ ]:
#  treino do modelo antigo 

X_train = X_train.reshape((-1, X_train.shape[1], X_train.shape[2], 1))
print(X_train.shape[0])
# history = model.fit(X_train,y_train,epochs=35, validation_data=(X_validation, y_validation))
history = model.fit(X_train,y_train,epochs=35, validation_data=(X_test, y_test))

In [ ]:
# validação do modelo antigo

import matplotlib.pyplot as pyplot

print(history.history)

testError, testAccuracy = model.evaluate(X_test, y_test)
print("Accuracy on test set is:{}".format(testAccuracy))

pyplot.plot(history.history['accuracy'], label='train_accuracy')
pyplot.plot(history.history['val_accuracy'], label = 'val_accuracy')
pyplot.xlabel('Epoch')
pyplot.ylabel('Accuracy')
pyplot.ylim([0.1, 1.1])
pyplot.legend(loc='lower right')